# Run Individual Solvers for Data Collection

In [1]:
import modcma
from modcma import ModularCMAES
import numpy as np
import pandas as pd
from ioh import problem
import scipy.optimize
from pathlib import Path

import warnings
# Filter out unwanted termination criteria warnings from modcma
warnings.filterwarnings(module = "modcma", action = "ignore", message = "Termination criteria met.*")

In [2]:
class FuncWrapped:
    """
    Wrapper for optimization function for logging and early termination.
    """

    def __init__(self, func, yopt, log10_targets):
        self.fn = func
        self.fevals = 0

        self.targets = np.power(10, log10_targets)
        self.next_target = max(self.targets)

        self.hits = np.repeat(np.inf, len(self.targets))
        self.unsolved_targets = sum(self.hits == np.inf)

        self.yopt = yopt

    def __call__(self, x):
        if self.unsolved_targets == 0:
            return self.yopt

        self.fevals += 1
        fi = self.fn(x)

        gap = fi - self.yopt

        if gap < self.next_target:
            ts = np.logical_and(gap <= self.targets, self.hits == np.inf)
            self.hits[ts] = self.fevals

            self.unsolved_targets = sum(self.hits == np.inf)
            if self.unsolved_targets > 0:
                self.next_target = max(self.targets[self.hits == np.inf])
            else:
                self.next_target = 0
        
        return fi
    
    def get_hits_evals(self):
        return (self.hits, self.fevals)

In [3]:
def run_scipy(params, func, yopt, log10_targets):
    func_wrapped = FuncWrapped(func, yopt, log10_targets)
    scipy.optimize.minimize(func_wrapped, x0 = np.repeat(0, len(func.optimum.x)), **params)

    hits, fevals = func_wrapped.get_hits_evals()

    solved = hits < np.inf
    evaluations = [int(min(hit, fevals)) for hit in hits]

    df = pd.DataFrame()
    df["log10_target"] = log10_targets
    df["solved"] = solved
    df["evaluations"] = evaluations

    return df

In [4]:
def run_cma(params, func, yopt, log10_targets):
    func_wrapped = FuncWrapped(func, yopt, log10_targets)
    
    dim = len(func.optimum.x)

    cma = modcma.ModularCMAES(func_wrapped, d = dim, budget = 100_000 * dim,
                              compute_termination_criteria = True, target = yopt + np.power(10, min(log10_targets)), **params)

    while not (any(cma.break_conditions) or ("local_restart" not in params.keys() and any(cma.parameters.termination_criteria.values()))):
        cma.step()

    hits, fevals = func_wrapped.get_hits_evals()

    solved = hits < np.inf
    evaluations = [int(min(hit, fevals)) for hit in hits]

    df = pd.DataFrame()
    df["log10_target"] = log10_targets
    df["solved"] = solved
    df["evaluations"] = evaluations

    return df

In [5]:
# Common setup
log10_targets = np.linspace(2, -8, 51)

# Common BBOB dimensions in 2-10
dims = [2,3,5,10]

# FIDs 1...24
fids = range(1, 25)

# IIDs 101...600
iids = range(101,601)

# One repetition per IID
repetitions = range(1, 2)

In [6]:
for dim in dims:
    
    budget = 100_000 * dim
    lambda_default = int(4 + np.floor(3 * np.log(dim)))

    cma_configurations = {
        # "bipop": {"local_restart": "BIPOP"},
        # "bipop-active": {"local_restart": "BIPOP", "active": True},
        "default": {},
        "2L": {"lambda_": 2 * lambda_default},
        "4L": {"lambda_": 4 * lambda_default},
        "8L": {"lambda_": 8 * lambda_default},
        "16L": {"lambda_": 16 * lambda_default},
        "32L": {"lambda_": 32 * lambda_default},
        "64L": {"lambda_": 64 * lambda_default},
        # "active": {"active": True},
        # "elitist": {"elitist": True},
        # "orthogonal": {"orthogonal": True},
        # # "sequential": {"sequential": True},
        # # "threshold_convergence": {"threshold_convergence": True},
        # # "base_sampler_sobol": {"base_sampler": "sobol"},
        # "mirrored": {"mirrored": "mirrored"},
        # # "mirrored_pairwise": {"mirrored": "mirrored pairwise"},
    }

    scipy_configurations = {
        "SLSQP": {"method": "SLSQP"},
        "L-BFGS-B": {"method": "L-BFGS-B"},
        "Powell": {"method": "Powell"},
    }

    configurations = cma_configurations | scipy_configurations

    for param_id, params in configurations.items():

        for fid in fids:
            dfs = []

            for iid in iids:
                print(f"FID: {fid}, IID: {iid}, ParamID: {param_id:<15}", end = "\r")
                # print(f"FID: {fid}, IID: {iid}", end = "\r")

                for rep in repetitions:
                    np.random.seed(1_000_000 * fid + 1_000 * iid + rep)

                    func = problem.BBOB.create(problem_id = fid, instance_id = iid, dimension = dim)
                    
                    if param_id in scipy_configurations.keys():
                        df = run_scipy(params, func, func.optimum.y, log10_targets)
                    else:
                        df = run_cma(params, func, func.optimum.y, log10_targets)
                    
                    df["fid"] = fid
                    df["iid"] = iid
                    df["rep"] = rep
                    df["dim"] = dim
                    df["param_id"] = param_id

                    dfs.append(df)

            fid_df = pd.concat(dfs)
            csv_path = Path(f"output-500/{dim}D/{param_id}/F{fid}_D{dim}.csv")
            if not csv_path.parent.exists():
                csv_path.parent.mkdir(parents = True)
            fid_df.to_csv(csv_path, float_format="%.1f", index = False)

/Users/lennartschapermeier/source/research/cma-rsmp/.conda/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


/Users/lennartschapermeier/source/research/cma-rsmp/.conda/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


/Users/lennartschapermeier/source/research/cma-rsmp/.conda/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


# Test Individual CMA-ES Run

In [84]:
budget = 100_000 * dim

np.random.seed(0xC0FFEE)

# cma = AskTellCMAES(dim, lambda_ = 640, budget=budget, compute_termination_criteria = True)
cma = ModularCMAES(func, d = dim, lambda_ = 640, budget=budget, compute_termination_criteria = True)

targets = np.power(10, np.linspace(2, -8, 51))
hit = np.repeat(np.inf, 51)

i = 0
best_gap = np.inf

while not (any(cma.break_conditions) or any(cma.parameters.termination_criteria.values())):
   # i += 1
   
   # # Retrieve a single new candidate solution
   # xi = cma.ask()

   # # Evaluate the objective function
   # fi = func(np.transpose(xi))[0]
   # # Update the algorithm with the objective function value
   # cma.tell(xi, fi)

   # gap = fi - func.optimum.y

   # if (gap < best_gap):
   #    ts = np.logical_and(gap <= targets, np.logical_not(best_gap < targets))
   #    hit[ts] = i
   #    best_gap = gap

   cma.step()

print(f"eval: {cma.parameters.used_budget}, delta: {cma.parameters.fopt - func.optimum.y}")

eval: 278400, delta: inf
